In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
import tensorflow.keras
from keras.models import Sequential, Model
from keras.layers import *
from keras.utils import Sequence
from keras.layers import Conv2D, MaxPooling2D
from qkeras import *

from keras.utils import Sequence
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping

import os
import random
from datetime import datetime
import time

pi = 3.14159265359

maxval=1e9
minval=1e-9

2025-06-17 18:45:12.498997: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-17 18:45:13.449144: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from dataloaders.OptimizedDataGenerator_v2 import OptimizedDataGenerator
from loss import *
from models import *

In [3]:
seed = 10
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
dataset_base_dir = "/data/dajiang/smart-pixels/dataset_3sr/shuffled/dataset_3sr_16x16_50x12P5_parquets/contained/"
tfrecords_base_dir = os.path.join(dataset_base_dir, "TFR_files", "2t")

dataset_train_dir = os.path.join(dataset_base_dir, "train")
dataset_validation_dir = os.path.join(dataset_base_dir, "test")
tfrecords_dir_train = os.path.join(tfrecords_base_dir, "TFR_train")
tfrecords_dir_val   = os.path.join(tfrecords_base_dir, "TFR_val")

In [5]:
print(f'Number of training files: {len(os.listdir(dataset_train_dir))}')
print(f'Number of validation files: {len(os.listdir(dataset_validation_dir))}')

Number of training files: 80
Number of validation files: 20


In [6]:
batch_size = 5000
val_batch_size = 5000
train_file_size = len(os.listdir(dataset_train_dir))
val_file_size = len(os.listdir(dataset_validation_dir))

In [7]:
start_time = time.time()
validation_generator = OptimizedDataGenerator(
    dataset_base_dir = dataset_validation_dir,
    file_type = "parquet",
    data_format = "3D",
    batch_size = val_batch_size,
    file_count = val_file_size,
    to_standardize= True,
    include_y_local= False,
    labels_list = ['x-midplane','y-midplane','cotAlpha','cotBeta'],
    input_shape = (2,16,16), # (20,16,16),
    transpose = (0,2,3,1),
    shuffle = False, 
    files_from_end=True,

    tfrecords_dir = tfrecords_dir_val,
    use_time_stamps = [0,19],
    max_workers = 2,
    #load_from_tfrecords_dir = tfrecords_dir_val
)

print("--- Validation generator %s seconds ---" % (time.time() - start_time))


Processing Files...: 100%|██████████| 20/20 [00:04<00:00,  4.20it/s]


Directory /data/dajiang/smart-pixels/dataset_3sr/shuffled/dataset_3sr_16x16_50x12P5_parquets/contained/TFR_files/2t/TFR_val is removed...


Saving batches as TFRecords: 100%|██████████| 21/21 [00:07<00:00,  2.90it/s]


Metadata saved successfully ast /data/dajiang/smart-pixels/dataset_3sr/shuffled/dataset_3sr_16x16_50x12P5_parquets/contained/TFR_files/2t/TFR_val/metadata.json
Loading metadata from /data/dajiang/smart-pixels/dataset_3sr/shuffled/dataset_3sr_16x16_50x12P5_parquets/contained/TFR_files/2t/TFR_val/metadata.json
--- Validation generator 12.389603614807129 seconds ---


In [8]:
# training generator
start_time = time.time()
training_generator = OptimizedDataGenerator(
    dataset_base_dir = dataset_train_dir,
    file_type = "parquet",
    data_format = "3D",
    batch_size = batch_size,
    file_count = train_file_size,
    to_standardize= True,
    include_y_local= False,
    labels_list = ['x-midplane','y-midplane','cotAlpha','cotBeta'],
    input_shape = (2,16,16), # (20,16,16),
    transpose = (0,2,3,1),
    shuffle = False, # True 

    tfrecords_dir = tfrecords_dir_train,
    use_time_stamps = [0,19],
    max_workers = 2,
    #load_from_tfrecords_dir = tfrecords_dir_train

)
print("--- Training generator %s seconds ---" % (time.time() - start_time))

Processing Files...: 100%|██████████| 80/80 [00:18<00:00,  4.30it/s]


Directory /data/dajiang/smart-pixels/dataset_3sr/shuffled/dataset_3sr_16x16_50x12P5_parquets/contained/TFR_files/2t/TFR_train is removed...


Saving batches as TFRecords: 100%|██████████| 84/84 [00:27<00:00,  3.08it/s]


Metadata saved successfully ast /data/dajiang/smart-pixels/dataset_3sr/shuffled/dataset_3sr_16x16_50x12P5_parquets/contained/TFR_files/2t/TFR_train/metadata.json
Loading metadata from /data/dajiang/smart-pixels/dataset_3sr/shuffled/dataset_3sr_16x16_50x12P5_parquets/contained/TFR_files/2t/TFR_train/metadata.json
--- Training generator 46.41184854507446 seconds ---


In [9]:
training_generator = OptimizedDataGenerator(
    load_from_tfrecords_dir = tfrecords_dir_train,
    shuffle = True,
    seed = seed,
    quantize = True
)

validation_generator = OptimizedDataGenerator(
    load_from_tfrecords_dir = tfrecords_dir_val,
    shuffle = True,
    seed = seed,
    quantize = True
)


Loading metadata from /data/dajiang/smart-pixels/dataset_3sr/shuffled/dataset_3sr_16x16_50x12P5_parquets/contained/TFR_files/2t/TFR_train/metadata.json


Loading metadata from /data/dajiang/smart-pixels/dataset_3sr/shuffled/dataset_3sr_16x16_50x12P5_parquets/contained/TFR_files/2t/TFR_val/metadata.json


In [10]:
model=CreateModel((16,16,2),n_filters=5,pool_size=3)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=custom_loss
)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 16, 16, 2)]       0         
                                                                 
 q_separable_conv2d (QSepar  (None, 14, 14, 5)         33        
 ableConv2D)                                                     
                                                                 
 q_activation (QActivation)  (None, 14, 14, 5)         0         
                                                                 
 q_conv2d (QConv2D)          (None, 14, 14, 5)         30        
                                                                 
 q_activation_1 (QActivatio  (None, 14, 14, 5)         0         
 n)                                                              
                                                                 
 average_pooling2d (Average  (None, 4, 4, 5)           0     

In [11]:
# training
pitch = '50x12P5'
fingerprint = '%08x' % random.randrange(16**8)
base_dir = '/home/dajiang/smart-pixels-ml/weights/dataset_3src_16x16_weights/'
weights_dir = base_dir + 'weights-{}-bs{}-{}-2t-checkpoints'.format(pitch, batch_size, fingerprint)

# create output directories
if os.path.isdir(base_dir):
    os.mkdir(weights_dir)
else:
    os.mkdir(base_dir)
    os.mkdir(weights_dir)
    
checkpoint_filepath = weights_dir + '/weights.{epoch:02d}-t{loss:.2f}-v{val_loss:.2f}.hdf5'
mcp = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    save_best_only=False,
)

print('Model fingerprint: {}'.format(fingerprint))

Model fingerprint: 34c2da80


In [ ]:
history = model.fit(x=training_generator,
                    validation_data=validation_generator,
                    callbacks=[mcp],
                    epochs=1000,
                    shuffle=False, # shuffling now occurs within the data-loader
                    verbose=1)

Epoch 1/1000


2025-06-17 18:46:32.197636: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8906
2025-06-17 18:46:32.312625: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-06-17 18:46:32.552624: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2025-06-17 18:46:32.558756: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0xb376ac0
2025-06-17 18:46:32.578990: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f5df0e4cb90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-06-17 18:46:32.579024: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB MIG 1g.5gb, Compute Capability 8.0
2025-06-17 18:46:32.583064: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir

84/84 [==============================] - 19s 178ms/step - loss: 62272.3984 - val_loss: 16089.8379
Epoch 2/1000
84/84 [==============================] - 7s 79ms/step - loss: 13463.6221 - val_loss: 10952.5928
Epoch 3/1000
84/84 [==============================] - 7s 78ms/step - loss: 8779.8193 - val_loss: 6302.3794
Epoch 4/1000
84/84 [==============================] - 7s 79ms/step - loss: 4569.5757 - val_loss: 2709.1526
Epoch 5/1000
84/84 [==============================] - 7s 79ms/step - loss: 1710.5869 - val_loss: 230.3340
Epoch 6/1000
84/84 [==============================] - 7s 80ms/step - loss: -890.5454 - val_loss: -1906.1260
Epoch 7/1000
84/84 [==============================] - 7s 78ms/step - loss: -2578.3140 - val_loss: -3366.5847
Epoch 8/1000
84/84 [==============================] - 6s 77ms/step - loss: -3990.9927 - val_loss: -4335.8901
Epoch 9/1000
84/84 [==============================] - 7s 78ms/step - loss: -3260.2598 - val_loss: -4234.2295
Epoch 10/1000
84/84 [=================